<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Web/blob/main/141-Llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LlamaIndex <> Langchain Integrations

This demo notebook shows how you can provide integrations between LlamaIndex and Langchain. It provides the following examples:
- Using LlamaIndex as a callable tool with a Langchain agent
- Using LlamaIndex as a memory module; this allows you to insert arbitrary amounts of conversation history with a Langchain chatbot!

In [1]:
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


In [ ]:
!pip install llama-index pypdf

In [ ]:
!pip install langchain langchain-core langchain-community

In [12]:
!pip install python-dotenv

In [13]:
!echo "OPENAI_API_KEY=sk-" >> .env

In [14]:
import openai, os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
# Access the API key using the variable name defined in the .env file
api_key = os.getenv("OPENAI_API_KEY")

In [15]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

#### Using LlamaIndex as a Callable Tool

In [16]:
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [8]:
!mkdir data

In [17]:
documents = SimpleDirectoryReader("./data").load_data()
index = VectorStoreIndex.from_documents(documents=documents)

In [18]:
tools = [
    Tool(
        name="LlamaIndex",
        func=lambda q: str(index.as_query_engine().query(q)),
        description="useful for when you want to answer questions about the author. The input to this tool should be a complete english sentence.",
        return_direct=True,
    ),
]

In [19]:
# set Logging to DEBUG for more detailed outputs
memory = ConversationBufferMemory(memory_key="chat_history")
llm = ChatOpenAI(temperature=0)
agent_executor = initialize_agent(
    tools, llm, agent="conversational-react-description", memory=memory
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [20]:
agent_executor.run(input="hi, i am bob")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'Hello Bob! How can I assist you today?'

In [ ]:
agent_executor.run(input="what's my name?")

#### Using LlamaIndex as a memory module

In [ ]:
# try using SummaryIndex!
from langchain.llms import OpenAI
from langchain.llms import OpenAIChat
from langchain.agents import initialize_agent

from llama_index.core import SummaryIndex
from llama_index.langchain_helpers.memory_wrapper import GPTIndexChatMemory

In [ ]:
index = SummaryIndex([])

In [ ]:
# set Logging to DEBUG for more detailed outputs
# NOTE: you can also use a conversational chain

memory = GPTIndexChatMemory(
    index=index,
    memory_key="chat_history",
    query_kwargs={"response_mode": "compact"},
    # return_source returns source nodes instead of querying index
    return_source=True,
    # return_messages returns context in message format
    return_messages=True,
)
# llm = OpenAIChat(temperature=0)
llm=OpenAI(temperature=0)
agent_executor = initialize_agent(
    [], llm, agent="conversational-react-description", memory=memory
)

In [ ]:
agent_executor.run(input="hi, i am bob")

'Hi Bob, nice to meet you! How can I help you today?'

In [ ]:
# NOTE: the query now calls the SummaryIndex memory module.
agent_executor.run(input="what's my name?")

'What is your name?'